<a href="https://colab.research.google.com/github/krishnakantsingh14/KaggleLiverpool/blob/master/kaggle_liverpool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LB score 0.938

In [0]:
import os 

In [0]:
PATH = '/content/drive/My Drive/liverpool_ion/'

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import *
import pandas as pd
import numpy as np
import random
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa
import gc

import tensorflow.keras as keras
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

import os

In [0]:
for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/My Drive/liverpool_ion/sample_submission.csv
/content/drive/My Drive/liverpool_ion/test.csv
/content/drive/My Drive/liverpool_ion/train.csv
/content/drive/My Drive/liverpool_ion/Y_test_proba.npy
/content/drive/My Drive/liverpool_ion/Y_train_proba.npy
/content/drive/My Drive/liverpool_ion/fourGPUlayers_using_batch0.h5
/content/drive/My Drive/liverpool_ion/fourGPUlayers_using_batch1.h5
/content/drive/My Drive/liverpool_ion/fourGPUlayers_using_batch2.h5
/content/drive/My Drive/liverpool_ion/fourGPUlayers_using_batch3.h5
/content/drive/My Drive/liverpool_ion/fourGPUlayers_using_batch4.h5
/content/drive/My Drive/liverpool_ion/submission_fourGPUlayers.csv
/content/drive/My Drive/liverpool_ion/_model_1_chk.h5
/content/drive/My Drive/liverpool_ion/_model_2_chk.h5
/content/drive/My Drive/liverpool_ion/_model_3_chk.h5
/content/drive/My Drive/liverpool_ion/modelFromKaggle3.h5
/content/drive/My Drive/liverpool_ion/_model_4_chk.h5
/content/drive/My Drive/liverpool_ion/modelFromKaggle

In [0]:
EPOCHS = 320
NNBATCHSIZE = 16
GROUP_BATCH_SIZE = 4000
SEED = 321
LR = 0.0015
SPLITS = 6

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [0]:
def lr_schedule(epoch):
    if epoch < 20:
        lr = LR
    elif epoch < 30:
        lr = LR / 3
    elif epoch < 40:
        lr = LR / 5
    elif epoch < 50:
        lr = LR / 7
    elif epoch < 60:
        lr = LR / 9
    elif epoch < 70:
        lr = LR / 11
    elif epoch < 80:
        lr = LR / 13
    else:
        lr = LR / 100
    return lr

In [0]:
def read_data():
    train = pd.read_csv(f'{PATH}/train.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv(f'{PATH}/test.csv', dtype={'time': np.float32, 'signal': np.float32})
    sub  = pd.read_csv(f'{PATH}/sample_submission.csv', dtype={'time': np.float32})
    train_o = pd.read_csv(f'{PATH}/train.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test_o  = pd.read_csv(f'{PATH}/test.csv', dtype={'time': np.float32, 'signal': np.float32})
    
    # Y_train_proba = np.load(f"{PATH}/Y_train_proba.npy")
    # Y_test_proba = np.load(f"{PATH}/Y_test_proba.npy")
    
    # for i in range(11):
    #     train[f"proba_{i}"] = Y_train_proba[:, i]
    #     test[f"proba_{i}"] = Y_test_proba[:, i]
    train['signal_o'] = train_o.signal
    test['signal_o'] = test_o.signal 
    del train_o, test_o

    return train, test, sub

In [0]:
def batching(df, batch_size):
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

In [0]:
def lag_with_pct_change(df, windows):
    for window in windows:    
        df['signal_shift_pos_' + str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' + str(window)] = df.groupby('group')['signal'].shift(-1 * window).fillna(0)
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
#     # create signal ** 2 (this is the new feature)
    df['signal_2'] = df['signal'] ** 2
    df['signal_o2'] = df['signal_o'] ** 2

    df['sigmoid'] = 1/(1+np.exp(-df['signal']))
    df['tanh'] = np.tanh(df['signal'])

    return df

In [0]:

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

In [0]:

def twolayerGRU(shape):
  input_ = keras.layers.Input(shape = shape)
  gru_h1 = tf.keras.layers.GRU(64, return_sequences=True)(input_)
  gru_h2 = tf.keras.layers.GRU(32, return_sequences=True)(gru_h1)
  output = tf.keras.layers.Dense(11, activation="softmax")(gru_h2)
  model = keras.Model(inputs=[input_], outputs=[output])
  opt = Adam(lr = LR)
  opt = tfa.optimizers.SWA(opt)
  model.compile(loss = tfa.losses.sigmoid_focal_crossentropy, optimizer = opt, metrics = ['accuracy'])
  return model

In [0]:
# def ClassifierModel(shape):
    
#     class GatedActivationUnit(keras.layers.Layer):

#         def __init__(self, activation="tanh", **kwargs):
#             super().__init__(**kwargs)
#             self.activation = keras.activations.get(activation)
    
#         def call(self, inputs):
#             n_filters = inputs.shape[-1] //2
#             linear_output = self.activation(inputs[...,:n_filters])
#             gate = keras.activations.sigmoid(inputs[...,n_filters:])
#             return self.activation(linear_output)*gate 

#     def wavenet_residual_block(inputs, n_filters, dilation_rate, padding ="same"):
#         z = keras.layers.Conv1D(2*n_filters, kernel_size =2, padding = padding,
#                             dilation_rate = dilation_rate)(inputs)
#         z = keras.layers.Conv1D(n_filters, kernel_size=1)(z)
#         return keras.layers.Add()([z, inputs]), z 
    
#     def cbr(x, out_layer, kernel, stride, dilation):
#         x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
#         x = BatchNormalization()(x)
#         x = Activation("relu")(x)
#         return x

#     n_layers_per_block = 10 #10 
#     n_blocks = 3 # 3 in the paper
#     n_filters = 64*2 # 128 in the paper
#     n_outputs = 11 # 256 in the paper

#     inputs = keras.layers.Input(shape=shape)

#     z = cbr(inputs, n_filters, 7, 1, 1)
#     z = keras.layers.BatchNormalization()(z)
#     z = cbr(z, n_filters, 5, 1, 1)
#     z = keras.layers.BatchNormalization()(z)

#     skip_to_last = []
#     for dilation_rate in [2**i for i in range(n_layers_per_block)] * n_blocks:
#         z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
#         skip_to_last.append(skip)
#     z = keras.activations.relu(keras.layers.Add()(skip_to_last))

#     z = keras.layers.BatchNormalization()(z)
#     # z = keras.layers.Conv1D(n_filters, kernel_size=5, activation="relu")(z)
#     z = cbr(z, n_filters, 1, 1, 1)
#     z = Dropout(0.2)(z)
#     Y_proba = tf.keras.layers.Dense(n_outputs, activation="softmax")(z)
    
    
#     model = keras.Model(inputs=[inputs], outputs=[Y_proba])
#     opt = Adam(lr = LR)
#     model.compile(loss = losses.CategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
#     return model

# def conVnet(shape):
#   model = keras.models.Sequential()
#   model.add(keras.layers.InputLayer(input_shape=shape))
#   for rate in (1, 2, 4, 8) * 2:
#     model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
#                                   activation="relu", dilation_rate=rate))
    
#   model.add(keras.layers.Dense(11, activation="softmax")(concat))
#   loss=tfa.losses.focal_loss()

#   model.compile(loss = losses.CategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
#   return model



# def fusedGruConv(shape):
#   input_ = keras.layers.Input(shape = shape)
#   gru_h1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True))(input_)
#   gru_h2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(gru_h1)
#   conv_h1  = tf.keras.layers.Conv1D(filters=64, kernel_size=4, padding="same")(input_)
#   conv_h2  = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv_h1)
#   concat = keras.layers.Concatenate()([gru_h2, conv_h2 ])
#   output = tf.keras.layers.Dense(11, activation="softmax")(concat)
#   model = keras.Model(inputs=[input_], outputs=[output])
#   opt = Adam(lr = LR)
# #     opt = tfa.optimizers.SWA(opt)
#   model.compile(loss = tfa.losses.sigmoid_focal_crossentropy, optimizer = opt, metrics = ['accuracy'])
#   return model

Old: 

    x = cbr(x, 64, 7, 1, 1)
    x = BatchNormalization()(x)
    x = wave_block(x, 16, 2, 12)
    x = BatchNormalization()(x)
    x = wave_block(x, 32, 2, 8)
    x = BatchNormalization()(x)
    x = wave_block(x, 64, 2, 4)
    x = BatchNormalization()(x)
    x = wave_block(x, 128, 2, 1)
    x = cbr(x, 64, 7, 1, 1)
    x = tf.keras.layers.GRU(64, return_sequences=True)(x)
    x = keras.layers.GRU(64 , return_sequences=True)(x)

In [0]:
# def modelFromKaggle(shape_):
    
#     def cbr(x, out_layer, kernel, stride, dilation):
#         x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
#         x = BatchNormalization()(x)
#         x = Activation("relu")(x)
#         return x
    
#     def wave_block(x, filters, kernel_size, n):
#         dilation_rates = [2**i for i in range(n)]
#         x = Conv1D(filters = filters,
#                    kernel_size = 1,
#                    padding = 'same')(x)
#         res_x = x
#         for dilation_rate in dilation_rates:
#             tanh_out = Conv1D(filters = filters,
#                               kernel_size = kernel_size,
#                               padding = 'same', 
#                               activation = 'tanh', 
#                               dilation_rate = dilation_rate)(x)
#             sigm_out = Conv1D(filters = filters,
#                               kernel_size = kernel_size,
#                               padding = 'same',
#                               activation = 'sigmoid', 
#                               dilation_rate = dilation_rate)(x)
#             x = Multiply()([tanh_out, sigm_out])
#             x = Conv1D(filters = filters,
#                        kernel_size = 1,
#                        padding = 'same')(x)
#             res_x = Add()([res_x, x])
#         return res_x
    
#     inp = Input(shape = (shape_))
#     x = tf.keras.layers.GRU(64, return_sequences=True)(inp)
#     x = cbr(x, 64, 7, 1, 1)
#     x = BatchNormalization()(x)
#     x = wave_block(x, 16, 2, 12)
#     x = BatchNormalization()(x)
#     x = wave_block(x, 32, 2, 8)
#     x = BatchNormalization()(x)
#     x = wave_block(x, 64, 2, 4)
#     x = BatchNormalization()(x)
#     x = wave_block(x, 128, 2, 1)
#     x = cbr(x, 64, 7, 1, 1)
#     x = tf.keras.layers.GRU(64, return_sequences=True)(x)
#     x = keras.layers.GRU(64 , return_sequences=True)(x)
#     # x = Dropout(0.2)(x)
#     out = Dense(11, activation = 'softmax', name = 'out')(x)
    
#     model = models.Model(inputs = inp, outputs = out)
    
#     opt = Adam(lr = LR)
#     opt = tfa.optimizers.SWA(opt)
#     model.compile(loss = losses.CategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
#     return model

In [0]:
# z = np.random.random((20,10,10))

In [0]:
#  tfa.losses.sigmoid_focal_crossentropy

In [0]:
# h1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True))
# h2  = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding="same")

In [0]:
# h1_out = h1(z)
# h2_out = h2(z) 

In [0]:
# tf.keras.layers.Concatenate()([h1_out, h2_out ]).shape

In [0]:
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis = 2).reshape(-1)
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis = 2).reshape(-1)
        score = f1_score(self.targets, pred, average = 'macro')
        print(f'F1 Macro Score: {score:.5f}')

In [0]:
def getCallbacks():
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=20, 
        verbose=1, restore_best_weights=True)

    return [early_stopping]
  


In [0]:
train, test, sample_submission = read_data()
train, test = normalize(train, test)
print('Reading and Normalizing Data Completed')
        
print('Creating Features')
print('Feature Engineering Started...')
train = run_feat_engineering(train, batch_size = GROUP_BATCH_SIZE)  
test = run_feat_engineering(test, batch_size = GROUP_BATCH_SIZE)
train, test, features = feature_selection(train, test)



Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...


In [0]:
train.shape

(5000000, 29)

In [0]:
len(features)

26

In [0]:
train.head()

,time,signal,open_channels,signal_o,group,signal_shift_pos_1,signal_shift_neg_1,signal_shift_pos_2,signal_shift_neg_2,signal_shift_pos_3,signal_shift_neg_3,signal_shift_pos_4,signal_shift_neg_4,signal_shift_pos_5,signal_shift_neg_5,signal_shift_pos_6,signal_shift_neg_6,signal_shift_pos_7,signal_shift_neg_7,signal_shift_pos_8,signal_shift_neg_8,signal_shift_pos_9,signal_shift_neg_9,signal_shift_pos_10,signal_shift_neg_10,signal_2,signal_o2,sigmoid,tanh
0,0.0001,-1.148772,0,-2.760655,0,0.000000,-1.184075,0.000000,-1.012891,0.000000,-1.298012,0.000000,-1.303999,0.0,-1.104036,0.0,-1.123085,0.0,-1.082287,0.0,-1.111547,0.0,-1.149429,0.0,-1.286463,1.319677,7.621218,0.240713,-0.817347
1,0.0002,-1.184075,0,-2.848034,0,-1.148772,-1.012891,0.000000,-1.298012,0.000000,-1.303999,0.000000,-1.104036,0.0,-1.123085,0.0,-1.082287,0.0,-1.111547,0.0,-1.149429,0.0,-1.286463,0.0,-1.096262,1.402034,8.111297,0.234320,-0.828732
2,0.0003,-1.012891,0,-2.424341,0,-1.184075,-1.298012,-1.148772,-1.303999,0.000000,-1.104036,0.000000,-1.123085,0.0,-1.082287,0.0,-1.111547,0.0,-1.149429,0.0,-1.286463,0.0,-1.096262,0.0,-1.137421,1.025949,5.877428,0.266414,-0.766955
3,0.0004,-1.298012,0,-3.130037,0,-1.012891,-1.303999,-1.184075,-1.104036,-1.148772,-1.123085,0.000000,-1.082287,0.0,-1.111547,0.0,-1.149429,0.0,-1.286463,0.0,-1.096262,0.0,-1.137421,0.0,-1.204464,1.684836,9.797131,0.214500,-0.861211
4,0.0005,-1.303999,0,-3.144854,0,-1.298012,-1.104036,-1.012891,-1.123085,-1.184075,-1.082287,-1.148772,-1.111547,0.0,-1.149429,0.0,-1.286463,0.0,-1.096262,0.0,-1.137421,0.0,-1.204464,0.0,-1.155627,1.700413,9.890105,0.213493,-0.862749


In [0]:
oof_ = np.zeros((len(train), 11)) # build out of folds matrix with 11 columns, they represent our target variables classes (from 0 to 10)
preds_ = np.zeros((len(test), 11))
target = ['open_channels']
group = train['group']
kf = GroupKFold(n_splits=5)
splits = [x for x in kf.split(train, train[target], group)]
new_splits = []
if not os.path.exists('Models'):
    os.makedirs('Models')
    
for sp in splits:
    new_split = []
    new_split.append(np.unique(group[sp[0]]))
    new_split.append(np.unique(group[sp[1]]))
    new_split.append(sp[1])    
    new_splits.append(new_split)

In [0]:


feats= features  
tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]], axis=1)

tr.columns = ['target_'+str(i) for i in range(11)] + ['group']
target_cols = ['target_'+str(i) for i in range(11)]
train_tr = np.array(list(tr.groupby('group').apply(lambda x: x[target_cols].values))).astype(np.float32)
train = np.array(list(train.groupby('group').apply(lambda x: x[feats].values)))
test = np.array(list(test.groupby('group').apply(lambda x: x[feats].values)))



In [0]:
nn_epochs = 80
nn_batch_size = 16

In [0]:
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [30]:
for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(new_splits[0:], start=0):
    train_x, train_y = train[tr_idx], train_tr[tr_idx]
    valid_x, valid_y = train[val_idx], train_tr[val_idx]
    print(f'Our training dataset shape is {train_x.shape}')
    print(f'Our validation dataset shape is {valid_x.shape}')

    gc.collect()
    shape_ = (None, train_x.shape[2])  # input is going to be the number of feature we are using (dimension 2 of 0, 1, 2)
    print (shape_)
    model = twolayerGRU(shape_)
        
        # using our lr_schedule function
    cb_lr_schedule = LearningRateScheduler(lr_schedule)
    chk_point = keras.callbacks.ModelCheckpoint(
        filepath= f'{PATH}/model_twolayersGRU_{n_fold}_chk.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', verbose=1)
    # onecycle = OneCycleScheduler(len(train_x) // nn_batch_size * nn_epochs, max_rate=0.05)
    call_backs = [ cb_lr_schedule, MacroF1(model, train_x, train_y),  MacroF1(model, valid_x, valid_y), chk_point ] + getCallbacks()
    model.fit(train_x,train_y,
              epochs = nn_epochs,
              callbacks = call_backs, # adding custom evaluation metric for each epoch
              batch_size = nn_batch_size,verbose = 2,
              validation_data = (valid_x,valid_y))
    model.save_weights(f'{PATH}/twoLayersgrus{n_fold}.h5')
        
    preds_f = model.predict(valid_x)
    f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  np.argmax(preds_f, axis=2).reshape(-1), average = 'macro') # need to get the class with the biggest probability
    print(f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}')
    preds_f = preds_f.reshape(-1, preds_f.shape[-1])
    oof_[val_orig_idx,:] += preds_f
    te_preds = model.predict(test)
    te_preds = te_preds.reshape(-1, te_preds.shape[-1])           
    preds_ += te_preds / SPLITS
    # calculate the oof macro f1_score
f1_score_ = f1_score(np.argmax(train_tr, axis = 2).reshape(-1),  np.argmax(oof_, axis = 1), average = 'macro') # axis 2 for the 3 Dimension array and axis 1 for the 2 Domension Array (extracting the best class)
print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')
sample_submission['open_channels'] = np.argmax(preds_, axis = 1).astype(int)
sample_submission.to_csv(f'{PATH}/twoayersGRU.csv', index=False, float_format='%.4f')

Our training dataset shape is (1000, 4000, 26)
Our validation dataset shape is (250, 4000, 26)
(None, 26)
Epoch 1/80
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
F1 Macro Score: 0.64446
F1 Macro Score: 0.64298

Epoch 00001: val_loss improved from inf to 0.11945, saving model to /content/drive/My Drive/liverpool_ion//model_twolayersGRU_0_chk.h5
63/63 - 61s - loss: 0.2166 - accuracy: 0.6521 - val_loss: 0.1194 - val_accuracy: 0.8116 - lr: 0.0015
Epoch 2/80
F1 Macro Score: 0.77108
F1 Macro Score: 0.76874

Epoch 00002: val_loss improved from 0.11945 to 0.06581, saving model to /content/drive/My Drive/liverpool_ion//model_twolayersGRU_0_chk.h5
63/63 - 60s - loss: 0.0832 - accuracy: 0.8722 - val_loss: 0.0658 - val_accuracy: 0.9044 - lr: 0.0015
Epoch 3/80
F1 Macro Score: 0.83118
F1 Macro Score: 0.82868

Epoch 00003: val_loss improved from 0.06581 to 0.04145, saving model to /content/drive/My Drive/liverpool_ion//model_twolayersGRU_0_chk.h5
63/63 - 60s - loss

In [0]:
f'{PATH}_model_{n_fold}'

'/content/drive/My Drive/liverpool_ion/_model_0'